<a href="https://colab.research.google.com/github/uoahy-6uoas/HG-Machine/blob/main/hg3-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**특성공학과 규제**
> 회귀당 하나의 종속 변수만 있을 수 있지만 독립 변수는 여러 개가 있을 수 있습니다. 이를 일반적으로 **다중 회귀** ~~다항회귀 아님~~라고 합니다. _TIBCO_

> **특성공학**은 데이터의 특성을 다루는 기술 [ex)농어의길이*두께-->새로운 특성]

In [1]:
import pandas as pd
df = pd.read_csv('https://bit.ly/perch_csv_data')
perch_full = df.to_numpy()

In [2]:
import numpy as np
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0,
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0,
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0,
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0,
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0,
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0,
     1000.0, 1000.0]
     )

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(perch_full,perch_weight)

In [4]:
# transformer of sklearn = 특성을 만들거나 전처리하기 위함
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit([[2,3]])
poly.transform([[2,3]])

array([[1., 2., 3., 4., 6., 9.]])

**특성공학**
[2,3]은 2개의 특성을 가진 것,<br>
polynomial_feature은 각 특성을 제곱한 값과 절편(=제곱하지 않는 값)을 반환<br>
절편을 특성으로 안쓰려면(이미 sklearn linear model이 절편을 제공하니) include_bias=False 사용하면 됨

In [5]:
#이걸 train셋에 적용해 보자
poly = PolynomialFeatures(include_bias=False) #PolynomialFeatures(degree=N) N값으로 제곱값 조정할 수 있음
poly.fit(train_input) #train_input = (n,3)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input) #fit은 훈련세트를 기준으로 변환 해야함.. 이유는 어디 써두신거지.
poly.get_feature_names_out() #어떤 특성으로 변했는지 확인

array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2'], dtype=object)

In [6]:
#훈련시작
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, train_target)
print(f"train_score : {lr.score(train_poly,train_target)}")
print(f"test_score : {lr.score(test_poly,test_target)}")

train_score : 0.9895251845550863
test_score : 0.9829533900270728


degree의 N값이 커지면 과대적합 될 가능성이 매우 큼 --> 적절한 N값으로 **규제** 필요

In [7]:
#안정된 데이터를 위해 정규화 과정을 거치자
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

In [8]:
#릿지회귀
from sklearn.linear_model import Ridge
ridge = Ridge() #Ridge(alpha=N) alpha는 얼마나 규제할 것인지의 정도를 결정함, 클수록 규제더함 = 과소적합 유도
ridge.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target)," : train score")
print(ridge.score(test_scaled, test_target)," : test score")

0.9852688482523752  : train score
0.9846971233604201  : test score


In [10]:
#라쏘회귀
from sklearn.linear_model import Lasso
lasso = Lasso()
lasso.fit(train_scaled, train_target)
print(lasso.score(train_scaled, train_target), " : train score")
print(lasso.score(test_scaled, test_target), " : test score")

0.9867404664620958  : train score
0.9842305279356575  : test score


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.915e+02, tolerance: 4.982e+02
  model = cd_fast.enet_coordinate_descent(


**확인문제**
1. a,b,c특성으로 이루어진 훈련세트를 PolynomialFeatures(degree=3)으로 변환했다. 데이터에 포함되는 특성은 = 1,a,a^2,a^3,a*b....
2. 특성을 표준화하는 클래스 = StandardScaler
3. 과대적합과 과소적합을 올바르게 표현한 것은 <br>
과대적합 : 테스트점수가 낮음, 과소적합 : 테스트점수가 높음 (일반적)